In [45]:
import pandas as pd
import numpy as np

import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb

from sklearn.metrics import roc_auc_score, accuracy_score

In [38]:
df = pd.read_csv('../data/train_dnn.csv', index_col=0)
df.columns = df.columns.str.strip()

In [39]:
def preprocess_data(df):
    df2 = df.copy()
    
    # del df2['BUTTER']
    # df2['gamma_PT'] = np.log(df2['gamma_PT'])
    # df2['B_DIRA_OWNPV'] = np.log(df2['B_DIRA_OWNPV'])
    # df2['Kst_892_0_IP_OWNPV'] = np.log(df2['Kst_892_0_IP_OWNPV'])
    
    return df2
    
df_train = preprocess_data(df)
X = df_train[df_train.columns[:-1]]
y = df_train[df_train.columns[-1]]

X

,B_OWNPV_CHI2,B_IPCHI2_OWNPV,B_FDCHI2_OWNPV,B_DIRA_OWNPV,B_PT,Kst_892_0_IP_OWNPV,Kst_892_0_cosThetaH,Kplus_IP_OWNPV,Kplus_P,piminus_IP_OWNPV,piminus_P,gamma_PT,piminus_ETA,Kplus_ETA,DNN
5,3.900054,1.868643,1990.246396,-6.309357e-07,9816.501198,-0.184687,0.072420,0.657268,79061.916645,1.427286,23416.733820,8.504832,3.448270,3.660611,0.070759
14,81.058899,1.537496,324.763824,-1.796860e-05,9583.550551,-1.474992,0.411845,0.241325,10827.134958,0.249671,17061.008829,8.919594,3.187614,3.308458,0.006111
20,40.704880,0.160716,14466.426007,-1.129311e-07,9139.378010,-1.391002,-0.737513,0.224783,33281.686307,0.808715,5450.649661,8.445245,2.969432,2.805554,0.263531
23,15.264702,3.412344,4633.426621,-3.092885e-07,9106.538742,0.141046,-0.089437,1.025922,82074.746937,1.613883,24533.543687,8.686373,3.875059,3.844981,0.995718
29,24.653332,0.549762,270.693850,-4.086086e-06,8473.149479,-1.465485,0.009472,0.252870,37655.239776,0.223891,25934.111565,8.481498,3.509980,3.232011,0.375655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212614,34.768871,0.111745,6934.611092,-2.459527e-07,11699.657257,0.050608,0.668364,0.951743,22688.546711,1.162431,27335.692429,8.803033,2.901365,2.947586,0.742070
212617,31.850922,0.487066,92.352673,-1.491943e-05,7363.659269,-1.436195,0.843792,0.200929,26170.078594,0.255193,48456.244039,8.578473,4.335271,4.206345,0.014528
212625,37.395596,6.594063,1306.870929,-1.173181e-05,5987.794148,-0.653630,0.104638,0.617412,43664.114278,0.420144,40228.867749,8.254526,4.010751,3.776591,0.050616
212636,25.520590,5.740875,850.607595,-4.027951e-05,7144.882924,-0.738505,0.600473,0.368414,14108.597057,0.673249,8368.403167,8.637242,2.638793,3.117137,0.072399


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

X_train = pd.DataFrame(data=scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_train.columns)

In [62]:
xgb_params = {
    'scale_pos_weight': 2,
    'learning_rate': 0.01,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'reg_alpha': 0.3,
    'max_depth': 6,
    'gamma': 10
}

model = xgb.XGBClassifier(**xgb_params)
model.fit(X_train, y_train, eval_set=[(X_train,y_train), (X_test,y_test)], eval_metric=['auc', 'error'], verbose=True)

[0]	validation_0-auc:0.73866	validation_0-error:0.34547
[1]	validation_0-auc:0.79202	validation_0-error:0.29454
[2]	validation_0-auc:0.79810	validation_0-error:0.28739
[3]	validation_0-auc:0.86563	validation_0-error:0.22330
[4]	validation_0-auc:0.86243	validation_0-error:0.22961
[5]	validation_0-auc:0.85776	validation_0-error:0.23638
[6]	validation_0-auc:0.85686	validation_0-error:0.23482
[7]	validation_0-auc:0.85124	validation_0-error:0.23859
[8]	validation_0-auc:0.87172	validation_0-error:0.21564
[9]	validation_0-auc:0.86742	validation_0-error:0.22015
[10]	validation_0-auc:0.86347	validation_0-error:0.22420
[11]	validation_0-auc:0.87606	validation_0-error:0.21000
[12]	validation_0-auc:0.88377	validation_0-error:0.20050
[13]	validation_0-auc:0.88047	validation_0-error:0.20445
[14]	validation_0-auc:0.87937	validation_0-error:0.20586
[15]	validation_0-auc:0.87615	validation_0-error:0.20990
[16]	validation_0-auc:0.87455	validation_0-error:0.21249
[17]	validation_0-auc:0.87231	validation_

[144]	validation_0-auc:0.89406	validation_0-error:0.19293
[145]	validation_0-auc:0.89397	validation_0-error:0.19251
[146]	validation_0-auc:0.89387	validation_0-error:0.19288
[147]	validation_0-auc:0.89420	validation_0-error:0.19251
[148]	validation_0-auc:0.89453	validation_0-error:0.19222
[149]	validation_0-auc:0.89446	validation_0-error:0.19213
[150]	validation_0-auc:0.89477	validation_0-error:0.19170
[151]	validation_0-auc:0.89471	validation_0-error:0.19166
[152]	validation_0-auc:0.89502	validation_0-error:0.19152
[153]	validation_0-auc:0.89495	validation_0-error:0.19156
[154]	validation_0-auc:0.89489	validation_0-error:0.19156
[155]	validation_0-auc:0.89517	validation_0-error:0.19123
[156]	validation_0-auc:0.89515	validation_0-error:0.19152
[157]	validation_0-auc:0.89508	validation_0-error:0.19133
[158]	validation_0-auc:0.89502	validation_0-error:0.19142
[159]	validation_0-auc:0.89530	validation_0-error:0.19123
[160]	validation_0-auc:0.89522	validation_0-error:0.19161
[161]	validati

[286]	validation_0-auc:0.90130	validation_0-error:0.18653
[287]	validation_0-auc:0.90129	validation_0-error:0.18653
[288]	validation_0-auc:0.90128	validation_0-error:0.18649
[289]	validation_0-auc:0.90138	validation_0-error:0.18635
[290]	validation_0-auc:0.90138	validation_0-error:0.18644
[291]	validation_0-auc:0.90136	validation_0-error:0.18653
[292]	validation_0-auc:0.90133	validation_0-error:0.18653
[293]	validation_0-auc:0.90134	validation_0-error:0.18658
[294]	validation_0-auc:0.90144	validation_0-error:0.18663
[295]	validation_0-auc:0.90152	validation_0-error:0.18649
[296]	validation_0-auc:0.90164	validation_0-error:0.18639
[297]	validation_0-auc:0.90162	validation_0-error:0.18644
[298]	validation_0-auc:0.90170	validation_0-error:0.18658
[299]	validation_0-auc:0.90170	validation_0-error:0.18649
[300]	validation_0-auc:0.90169	validation_0-error:0.18653
[301]	validation_0-auc:0.90178	validation_0-error:0.18653
[302]	validation_0-auc:0.90186	validation_0-error:0.18625
[303]	validati

[428]	validation_0-auc:0.90463	validation_0-error:0.18390
[429]	validation_0-auc:0.90463	validation_0-error:0.18381
[430]	validation_0-auc:0.90463	validation_0-error:0.18390
[431]	validation_0-auc:0.90465	validation_0-error:0.18390
[432]	validation_0-auc:0.90464	validation_0-error:0.18385
[433]	validation_0-auc:0.90467	validation_0-error:0.18390
[434]	validation_0-auc:0.90470	validation_0-error:0.18381
[435]	validation_0-auc:0.90469	validation_0-error:0.18390
[436]	validation_0-auc:0.90472	validation_0-error:0.18390
[437]	validation_0-auc:0.90475	validation_0-error:0.18399
[438]	validation_0-auc:0.90475	validation_0-error:0.18404
[439]	validation_0-auc:0.90475	validation_0-error:0.18404
[440]	validation_0-auc:0.90478	validation_0-error:0.18423
[441]	validation_0-auc:0.90479	validation_0-error:0.18423
[442]	validation_0-auc:0.90479	validation_0-error:0.18418
[443]	validation_0-auc:0.90482	validation_0-error:0.18428
[444]	validation_0-auc:0.90482	validation_0-error:0.18418
[445]	validati

[570]	validation_0-auc:0.90607	validation_0-error:0.18376
[571]	validation_0-auc:0.90607	validation_0-error:0.18385
[572]	validation_0-auc:0.90607	validation_0-error:0.18385
[573]	validation_0-auc:0.90607	validation_0-error:0.18390
[574]	validation_0-auc:0.90609	validation_0-error:0.18395
[575]	validation_0-auc:0.90609	validation_0-error:0.18390
[576]	validation_0-auc:0.90609	validation_0-error:0.18395
[577]	validation_0-auc:0.90609	validation_0-error:0.18390
[578]	validation_0-auc:0.90611	validation_0-error:0.18399
[579]	validation_0-auc:0.90612	validation_0-error:0.18399
[580]	validation_0-auc:0.90612	validation_0-error:0.18395
[581]	validation_0-auc:0.90612	validation_0-error:0.18395
[582]	validation_0-auc:0.90614	validation_0-error:0.18395
[583]	validation_0-auc:0.90615	validation_0-error:0.18399
[584]	validation_0-auc:0.90616	validation_0-error:0.18395
[585]	validation_0-auc:0.90616	validation_0-error:0.18395
[586]	validation_0-auc:0.90617	validation_0-error:0.18395
[587]	validati

[712]	validation_0-auc:0.90678	validation_0-error:0.18352
[713]	validation_0-auc:0.90678	validation_0-error:0.18352
[714]	validation_0-auc:0.90679	validation_0-error:0.18371
[715]	validation_0-auc:0.90679	validation_0-error:0.18366
[716]	validation_0-auc:0.90679	validation_0-error:0.18371
[717]	validation_0-auc:0.90680	validation_0-error:0.18371
[718]	validation_0-auc:0.90680	validation_0-error:0.18371
[719]	validation_0-auc:0.90680	validation_0-error:0.18362
[720]	validation_0-auc:0.90681	validation_0-error:0.18357
[721]	validation_0-auc:0.90681	validation_0-error:0.18357
[722]	validation_0-auc:0.90682	validation_0-error:0.18357
[723]	validation_0-auc:0.90682	validation_0-error:0.18366
[724]	validation_0-auc:0.90682	validation_0-error:0.18366
[725]	validation_0-auc:0.90683	validation_0-error:0.18366
[726]	validation_0-auc:0.90682	validation_0-error:0.18366
[727]	validation_0-auc:0.90683	validation_0-error:0.18366
[728]	validation_0-auc:0.90684	validation_0-error:0.18362
[729]	validati

[854]	validation_0-auc:0.90712	validation_0-error:0.18310
[855]	validation_0-auc:0.90712	validation_0-error:0.18310
[856]	validation_0-auc:0.90713	validation_0-error:0.18315
[857]	validation_0-auc:0.90713	validation_0-error:0.18319
[858]	validation_0-auc:0.90713	validation_0-error:0.18315
[859]	validation_0-auc:0.90713	validation_0-error:0.18315
[860]	validation_0-auc:0.90713	validation_0-error:0.18301
[861]	validation_0-auc:0.90713	validation_0-error:0.18301
[862]	validation_0-auc:0.90713	validation_0-error:0.18305
[863]	validation_0-auc:0.90713	validation_0-error:0.18315
[864]	validation_0-auc:0.90713	validation_0-error:0.18315
[865]	validation_0-auc:0.90713	validation_0-error:0.18319
[866]	validation_0-auc:0.90714	validation_0-error:0.18310
[867]	validation_0-auc:0.90714	validation_0-error:0.18315
[868]	validation_0-auc:0.90714	validation_0-error:0.18305
[869]	validation_0-auc:0.90714	validation_0-error:0.18301
[870]	validation_0-auc:0.90714	validation_0-error:0.18296
[871]	validati

[996]	validation_0-auc:0.90732	validation_0-error:0.18282
[997]	validation_0-auc:0.90732	validation_0-error:0.18286
[998]	validation_0-auc:0.90732	validation_0-error:0.18286
[999]	validation_0-auc:0.90732	validation_0-error:0.18286


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=10, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=2, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [63]:
y_hat_train = model.predict_proba(X_train)[:,1]
y_hat_test = model.predict_proba(X_test)[:,1]

print("AUC Train\t", roc_auc_score(y_train, y_hat_train))
print("AUC Test\t", roc_auc_score(y_test, y_hat_test))

AUC Train	 0.9202166818214954
AUC Test	 0.9073220870103504


In [60]:
lgb_params = {
    'num_leaves': 32,
    'min_data_in_leaf': 50,
    'objective': 'binary',
    'max_depth': -1,
    'learning_rate': 0.01,
    'boosting_type': "gbdt",
    'metric': 'logloss',
    'verbosity': 1,
    'feature_fraction': 0.8
}

dtrain = lgb.Dataset(X_train, label=y_train)
dtest = lgb.Dataset(X_test, label=y_test)

clf = lgb.train(lgb_params, dtrain, 10000, valid_sets=(dtrain, dtest), verbose_eval=200)

[LightGBM] [Info] Number of positive: 63901, number of negative: 127494
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010694 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 191395, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333870 -> initscore=-0.690734
[LightGBM] [Info] Start training from score -0.690734


In [61]:
y_hat_train = clf.predict(X_train)
y_hat_test = clf.predict(X_test)

print("AUC Train\t", roc_auc_score(y_train, y_hat_train))
print("AUC Test\t", roc_auc_score(y_test, y_hat_test))

AUC Train	 0.9537811534345763
AUC Test	 0.9069518957880283
